In [15]:
import pandas as pd
import os
import math
import nltk
import pickle
import json
from nltk.corpus import stopwords

from nltk.stem.snowball import SnowballStemmer
from collections import defaultdict
from sklearn.metrics.pairwise import cosine_similarity

In [16]:
#Load docs, queries and the inverted index
with open("Output/preprocessed_docs","rb") as file:
    docs = pickle.load(file)

with open("Output/preprocessed_queries","rb") as file:
    queries = pickle.load(file)

with open("Output/inverted_index","r") as file:
    inverted_index = json.load(file)


In [27]:
def query_weights1(tf,idf,term):
    return (0.5 + (0.5*tf[term])/tf[max(tf)]) * idf

In [33]:
def document_weights1(tf,idf,term):
    return tf*idf

In [36]:
def vector_space(query,docs, query_weight_func, doc_weight_func):

        tf={}
        
        for term in set(query):#Get query term frequency
                tf[term] = query.count(term)

        #Calculate query and doc weights
        query_weights = []
        document_weights = {}

        for doc in docs:
                document_weights[doc[0]] = []

        for term in set(query):

                if  len(inverted_index[term]):
                        
                        idf = math.log(len(docs)/len(inverted_index[term]))

                        query_weights.append(query_weight_func(tf,idf,term))

                        for doc in docs:

                                try:
                                        tf_doc = inverted_index[term][doc[0]]
                                except:
                                        tf_doc = 0

                                w = doc_weight_func(tf_doc,idf,term)
                                document_weights[doc[0]].append(w)   


        #Calcualte cosine similarity for each doc
        sim = {}
        for doc in document_weights:
                sim[doc] = cosine_similarity([query_weights],[document_weights[doc]])

        return sorted(sim.items(), key=lambda x:x[1])[-20:]

In [38]:
vector_space(queries[5],docs,query_weights1,document_weights1)

[('00950', array([[0.42380801]])),
 ('00546', array([[0.4248569]])),
 ('00430', array([[0.42739021]])),
 ('00592', array([[0.42856555]])),
 ('00100', array([[0.43080643]])),
 ('00189', array([[0.43363235]])),
 ('00333', array([[0.44618724]])),
 ('00195', array([[0.449021]])),
 ('01095', array([[0.45321368]])),
 ('00024', array([[0.45766177]])),
 ('00976', array([[0.45900989]])),
 ('01038', array([[0.46156412]])),
 ('00208', array([[0.46975672]])),
 ('00416', array([[0.50001282]])),
 ('00031', array([[0.54039761]])),
 ('00593', array([[0.54664244]])),
 ('00067', array([[0.55020713]])),
 ('00511', array([[0.55243399]])),
 ('00047', array([[0.60723291]])),
 ('00533', array([[0.65834519]]))]